## 4-qubit circuit for Grover's algorithm 

Below, we will implement the following 4-qubit circuit to implement Grover's algorithm, 
where we set the two marked states, $$|w> = |0101>$$

The initial state is $|\Psi_0> =|0000>$, and this state will change when passing each gate, which we will trace.

In [1]:
import numpy as np 
from IPython.display import Image
import itertools
import re

In [2]:
#Image(filename='/Users/gimoonnam/Desktop/ImplementationOfGroverAlgorithm/grover_3-qubit.png')

In [3]:
def define_basisFor1Q():
    vec1Q = {}
    vec1Q['0'] = np.array([[1],[0]])
    vec1Q['1'] = np.array([[0],[1]])
    return vec1Q

In [4]:
def construct_basisFor4Q():

    arr = np.array([['0','0','0','0'],['1','0','0','0'],
                ['1','1','0','0'],['1','1','1','0'],['1','1','1','1']])
    list_p =[]

    for row in arr:
        temp = itertools.permutations(row)  
        list_p.extend(set(tuple(x) for x in list(temp)))
        
    vec1q = define_basisFor1Q()    
    vec4q = {}

    for i in list_p:
        k = list(reversed(i))
        temp = vec1q[k[0]]
        for idx in range(1,len(k)):
            temp = np.kron(vec1q[k[idx]],temp) 
            
        state = i[0]+i[1]+i[2]+i[3]
        vec4q[state] = temp;  

    return vec4q



def DefineIndexTable4Qubit(vec4q):
    idxInVec4q = {}
    for key, vec in vec4q.items():
        idx = [i for i in range(vec.shape[0]) if vec[i]==1]
        idxInVec4q[idx[0]] = key
    return idxInVec4q 
    


In [5]:
# define gates 
# H for Hadamard gate 
# X for X gate 
# I for identity gate 

nqubit = 4
N = np.power(2,nqubit)

H = np.matrix('1 1; 1 -1')
X = np.matrix('0 1; 1 0')
Z = np.matrix('1 0; 0 -1')
H = H/np.sqrt(2)
I = np.matrix('1 0; 0 1')
I2 = np.kron(I,I)
I4 = np.kron(I,np.kron(I,I2))
H2 = np.kron(H,H)
H3 = np.kron(H,H2)
H4 = np.kron(H,H3)
X2 = np.kron(X,X)
X3 = np.kron(X,X2)
X4 = np.kron(X,X3)


vec4q = construct_basisFor4Q()
idxInVec4q = DefineIndexTable4Qubit(vec4q)


In [6]:
def ConvertMatrixToState(Psi):
    coeff = np.abs(Psi).max()
    Psi = Psi/coeff
    idx = [i for i in range(Psi.shape[0]) if np.abs(Psi[i])==1]
    vec = []
    for i in idx:
        if Psi[i] < 0: 
            vec.append('-' + '|' + idxInVec4q[i] + '>')
        else:
            vec.append('|' + idxInVec4q[i] + '>')
    vec.append('=> Prob = '+str(np.square(coeff)))
    print(vec)
    return None

# Constructing 4-qubit circuit with markded state $|\omega> = |0101>$


Initially, 
$$|\Psi_0> = |0000> $$




After Hadamard gates, the state changes with superpositioning of all possible states 

$$|\Psi_1> = \frac{1}{4} \left(|0000> + |0001>+ |0010>+ |0011>\\ + |0100>+ |0101>+ |0110>+ |0111> \\+ |1000>+ |1001>+ |1010>+ |1011> \\+ |1100>+ |1101>+ |1110>+ |1111>\right)$$


In [7]:
#initial state function is |0000> 
Psi_0 = vec4q['0000']

#After Hadamard gates, the state can be written as a sum of all possible states 
Psi_1 = np.matmul(H4,Psi_0)
ConvertMatrixToState(Psi_1)

['|0000>', '|0001>', '|0010>', '|0011>', '|0100>', '|0101>', '|0110>', '|0111>', '|1000>', '|1001>', '|1010>', '|1011>', '|1100>', '|1101>', '|1110>', '|1111>', '=> Prob = 0.06249999999999996']


## Designing Oracle Black Box that only flips the marked state 
$$U_f|\omega>=−|\omega> $$


In [8]:
Psi_2 = np.matmul(I4, Psi_1)
ConvertMatrixToState(Psi_2)

['|0000>', '|0001>', '|0010>', '|0011>', '|0100>', '|0101>', '|0110>', '|0111>', '|1000>', '|1001>', '|1010>', '|1011>', '|1100>', '|1101>', '|1110>', '|1111>', '=> Prob = 0.06249999999999996']


In [9]:
XI = np.kron(X,I)
XIXI = np.kron(X,np.kron(I,XI))
Psi_3 = np.matmul(XIXI,Psi_2)

cccZ = I4 
cccZ[N-1,N-1] = -1
Psi_4 = np.matmul(cccZ,Psi_3)
Psi_5 = np.matmul(XIXI,Psi_4)
ConvertMatrixToState(Psi_5)


['|0000>', '|0001>', '|0010>', '|0011>', '|0100>', '-|0101>', '|0110>', '|0111>', '|1000>', '|1001>', '|1010>', '|1011>', '|1100>', '|1101>', '|1110>', '|1111>', '=> Prob = 0.06249999999999996']


## Amplitude Amplification 

In [10]:
Psi_6 = np.matmul(H4,Psi_5)
Psi_7 = np.matmul(X4,Psi_6)
Psi_8 = np.matmul(cccZ,Psi_7)
Psi_9  = np.matmul(X4,Psi_8)
Psi_10 = np.matmul(H4,Psi_9)
ConvertMatrixToState(Psi_10)


['-|0101>', '=> Prob = 0.4726562499999991']
